In [43]:
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API"] = os.getenv("GROQ_API")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")

In [44]:
from langchain_groq import ChatGroq
model = ChatGroq(model = "Gemma2-9b-It" , api_key=os.getenv("GROQ_API")) 
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7ad450820770>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7ad4508223c0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [45]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hello, how can I assist you today?")])

AIMessage(content='As an AI assistant, I can help you with a variety of tasks, such as:\n\n* **Generating creative content:** I can write stories, poems, articles, and even code.\n* **Answering questions:** I have access to a vast store of knowledge and can answer your questions on a wide range of topics.\n* **Summarizing text:** I can condense large amounts of text into shorter, more manageable summaries.\n* **Translating languages:** I can translate text between multiple languages.\n* **Providing conversation:** I can engage in natural and meaningful conversations on a variety of topics.\n\nWhat can I do for you today?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 134, 'prompt_tokens': 18, 'total_tokens': 152, 'completion_time': 0.243636364, 'prompt_time': 0.00125165, 'queue_time': 0.189028675, 'total_time': 0.244888014}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logp

In [46]:

#basic example of history
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hello, i am mahdi and iam ai eng"),
        AIMessage(content="Hello ! its nice to meet you."),
        HumanMessage(content="what is my name? and what i do?")
    ]
    
)

AIMessage(content='You told me your name is Mahdi and that you are an AI engineer!  😄 \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 47, 'total_tokens': 80, 'completion_time': 0.06, 'prompt_time': 0.001641379, 'queue_time': 0.188437004, 'total_time': 0.061641379}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--ea657ede-80ab-4882-b35b-8680c4f383c5-0', usage_metadata={'input_tokens': 47, 'output_tokens': 33, 'total_tokens': 80})

### message history

In [47]:
from langchain_community.chat_message_histories import ChatMessageHistory 
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [48]:
store = {}
def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)


In [49]:
config = {"configurable":{"session_id": "user_session_1"}}  # the session information 


In [50]:
resposne = with_message_history.invoke(

        [HumanMessage(content="Hello, i am mahdi and iam ai eng")] ,
        config=config

)

In [51]:
resposne.content

"Hello Mahdi! It's nice to meet you. \n\nBeing an AI engineer is exciting! What kind of projects are you working on?  \n\nI'm eager to learn more about your work in the field of AI. 😊  \n\n"

In [52]:
config2 = {"configurable":{"session_id": "user_session_2"}}  # the session information 
response = with_message_history.invoke(

        [HumanMessage(content="Hello, what is my name?")] ,
        config=config2

)
response.content



"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to use it! 😊\n"

### Adding prompts

In [53]:
from langchain_core.prompts import ChatPromptTemplate , MessagesPlaceholder

In [54]:
prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
        ("system", "You are a helpful assistant.")
    ]
)

In [55]:
chain = prompt | model

In [56]:
chain.invoke({"messages":[HumanMessage(content="Hello, i am mahdi")]})

AIMessage(content="Hello Mahdi! 👋 \n\nIt's nice to meet you. How can I help you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 21, 'total_tokens': 49, 'completion_time': 0.050909091, 'prompt_time': 0.00133866, 'queue_time': 0.18845393, 'total_time': 0.052247751}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--5f7302e9-ec70-4553-91b7-7b228ec74a4b-0', usage_metadata={'input_tokens': 21, 'output_tokens': 28, 'total_tokens': 49})

In [57]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

config = {"configurable":{"session_id": "user_session_3"}}  # the session information
response = with_message_history.invoke([HumanMessage(content="Hello, my name is mahdi")], config=config)
response.content    


"Hello Mahdi, it's nice to meet you! \n\nHow can I be helpful today? 😄  \n\n"

In [58]:
## more complexity
prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
        ("system", "You are a helpful assistant , answer the following in this {language}")

    ]
)
chain = prompt | model

In [59]:
response = chain.invoke({"messages":[HumanMessage(content="Hello, i am mahdi")] , "language": "spainish"})
response.content    

'¡Hola Mahdi! 👋  \n\n¿Cómo estás?  😊 \n'

In [60]:
with_message_history= RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key= "messages"

)

In [61]:
config = { "configurable": { "session_id": "user_session_4" } }
response = with_message_history.invoke({"messages": [HumanMessage(content="Hello, my name is mahdi")], "language": "French"}, config=config)
response

AIMessage(content="Bonjour Mahdi !  😊 \n\nEnchanté(e) de te rencontrer !  \n\nComment puis-je vous aider aujourd'hui ? \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 28, 'total_tokens': 61, 'completion_time': 0.06, 'prompt_time': 0.001406289, 'queue_time': 0.188946702, 'total_time': 0.061406289}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--fee86fb7-e337-4626-9791-a9aae95395c6-0', usage_metadata={'input_tokens': 28, 'output_tokens': 33, 'total_tokens': 61})

# mamnaging history

In [62]:
from langchain_core.messages import SystemMessage , trim_messages

trimmer = trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on='human'

)
messages = [
        SystemMessage(content="You are a helpful assistant."),
        HumanMessage(content="Hello, i am john and iam ai eng"),
        AIMessage(content="Hello ! its nice to meet you."),
        HumanMessage(content=" i like ice cream"),
        AIMessage(content="i like ice cream too"),
        HumanMessage(content="i like chocolate"),
        AIMessage(content="i like chocolate too"),
        HumanMessage(content="i like strawberry"),
        AIMessage(content="i like strawberry too")
]


trimmer.invoke(
    messages
)


[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, i am john and iam ai eng', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello ! its nice to meet you.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content=' i like ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='i like ice cream too', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='i like chocolate', additional_kwargs={}, response_metadata={}),
 AIMessage(content='i like chocolate too', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='i like strawberry', additional_kwargs={}, response_metadata={}),
 AIMessage(content='i like strawberry too', additional_kwargs={}, response_metadata={})]

In [63]:
from operator import itemgetter 

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt 
    | model
)

result = chain.invoke({"messages": messages + [HumanMessage(content="What i like ?")] , "language": "en"})
result.content

"You like ice cream, chocolate, and strawberry! 🍦🍫🍓 \n\nIs there anything else you'd like to tell me about what you like? 😊 \n"

In [65]:
# lets add this to the messge history function

with_message_history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key= "messages"
)
config = {"configurable":{"session_id": "user_session_5"}}  # the session information

response = with_message_history.invoke({
    "messages": messages + [HumanMessage(content="What is my name ?")] ,
    "language": "en"
}, config=config)
response.content

"As a helpful assistant, I don't have access to personal information like your name.  \n\nWould you like to tell me your name? 😊 \n\n"